In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-project.org/b

In [2]:
# Start Spark session
# Spark session with an additional option that adds the driver to Spark:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big_Data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# download a Postgres driver that will allow Spark to interact with Postgres:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-10-01 05:15:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.95MB/s    in 0.2s    

2020-10-01 05:15:35 (4.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz'

In [5]:
spark.sparkContext.addFile(url)
sport_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Sports_v1_00.tsv.gz'), sep='\t', header=True)

In [6]:
clean_sport_df = sport_df.dropna().dropDuplicates()
clean_sport_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24002988|R1008599CB6XOO|B005J9TKZ8|     807909508|     NCAA Pillow Pet|          Sports|          5|            0|          0|   N|                Y|          Five Stars|    This gift rocks!| 2014-11-27|
|         US|   14266788|R100IR1LJNAK0E|B002IWXJ4W|     632337843|Magpul XTM Rail P...|          Sports|          5|    

In [7]:
# prepare to validate vine reviews by filtering for only verified purchases 
# to help filter out fake reviews
vine_verified_df = clean_sport_df.filter(sport_df['verified_purchase'] == 'Y').dropna().dropDuplicates()

In [8]:
vine_reviews_df = vine_verified_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'review_body'])
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|
+--------------+-----------+-------------+-----------+----+--------------------+
|R1008599CB6XOO|          5|            0|          0|   N|    This gift rocks!|
|R100IR1LJNAK0E|          5|            0|          0|   N|Great<br />  Well...|
|R100JTP9E01W14|          4|            1|          1|   N|Like others said ...|
|R100S0IZLWJDS4|          1|            5|          6|   N|We bought this gu...|
| R100U3Q12G58D|          1|            0|          0|   N|Size is at least ...|
|R1013PCJ63HJMO|          1|            2|          2|   N|These pants are t...|
|R101AE0OGK16A7|          5|            2|          2|   N|Sometime even the...|
|R101F7X1H5593W|          5|            0|          0|   N|Hello - I bought ...|
|R101RXI99MOWQY|          5|            0|          0|   N|My 9 year old son...|
|R1023SWPQVRTYV|          4|

In [9]:
vine_filter_df = vine_reviews_df.filter(vine_reviews_df['vine'] == 'Y')
vine_filter_df.show()

+--------------+-----------+-------------+-----------+----+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|
+--------------+-----------+-------------+-----------+----+--------------------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|
|R1H8QU9G9HLBWK|          5|           43|         49|   Y|The handy multi-p...|
|R2LO3B4TW53KY3|          5|            3|          3|   Y|This is a great p...|
|R3FYZFOAA7EZU9|          5|            1|          1|   Y|Before using Srix...|
|R20N8XGNA7X5EB|          5|

In [10]:
vine_filter_df.count()

11

In [12]:
from pyspark.sql.functions import mean, stddev
from pyspark.sql.functions import col
# vine_mean = vine_filter_df.select(mean(col('star_rating')))[0]
vine_mean = vine_filter_df.select(mean(col('star_rating'))).show()

+-----------------+
| avg(star_rating)|
+-----------------+
|4.818181818181818|
+-----------------+



In [13]:
# review lengths can be an indicator of veracity; spoofed reviews tend to be shorter
from pyspark.sql.functions import length
# Create a length column to be used as a future feature
review_length_df = vine_filter_df.withColumn('length', length(vine_filter_df['review_body']))
review_length_df.show()

+--------------+-----------+-------------+-----------+----+--------------------+------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|length|
+--------------+-----------+-------------+-----------+----+--------------------+------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|   329|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|   405|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|   824|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|  1915|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|   816|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|   628|
|R1H8QU9G9HLBWK|          5|           43|         49|   Y|The handy multi-p...|  5493|
|R2LO3B4TW53KY3|          5|            3|          3|   Y|This is a great p...|   888|
|R3FYZFOAA7EZU9|          5|    

In [14]:
review_length_df.describe().show()

+-------+--------------+-------------------+------------------+------------------+----+--------------------+------------------+
|summary|     review_id|        star_rating|     helpful_votes|       total_votes|vine|         review_body|            length|
+-------+--------------+-------------------+------------------+------------------+----+--------------------+------------------+
|  count|            11|                 11|                11|                11|  11|                  11|                11|
|   mean|          null|  4.818181818181818|               5.0|5.7272727272727275|null|                null|1290.1818181818182|
| stddev|          null|0.40451991747794536|12.672805529952713|14.402020060331182|null|                null|1550.5555661234341|
|    min|R1H8QU9G9HLBWK|                  4|                 0|                 0|   Y|Almost everyone i...|               265|
|    max| RUVM52QIHJ57R|                  5|                43|                49|   Y|This is a great p

In [15]:
# Add new column to hold word counts > 20
# typical rules on websites are to have reviews longer than 20 words
review_length_df.withColumn('above_avg_length', review_length_df['length']>20).show()

+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|length|above_avg_length|
+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|   329|            true|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|   405|            true|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|   824|            true|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|  1915|            true|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|   816|            true|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|   628|            true|
|R1H8QU9G9HLBWK|          5|           43|         49| 

In [16]:
# see number of reviews with greater than 20 words
review_length_df.filter(review_length_df.length >20).count()

11

In [17]:
long_reviews = review_length_df.filter(review_length_df.length >20).count()
total_reviews = review_length_df.count()

print(long_reviews/total_reviews)

1.0


In [18]:
# Add new column to hold word counts > mean
review_length_df.withColumn('above_avg_length', review_length_df['length']>1290).show()

+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|         review_body|length|above_avg_length|
+--------------+-----------+-------------+-----------+----+--------------------+------+----------------+
|R1R2FIKQC9XIKO|          4|            0|          1|   Y|I love Zumba!  It...|   329|           false|
|R2AK4UF93O4EGW|          5|            0|          1|   Y|My son is a hunte...|   405|           false|
|R2YCOOKA91SPQ4|          5|            1|          1|   Y|From the moment I...|   824|           false|
|R1ITLW7VCY8ZNV|          5|            2|          2|   Y|Almost everyone i...|  1915|            true|
| RUVM52QIHJ57R|          5|            1|          1|   Y|The Vanguard 62-I...|   816|           false|
|R2RVYNK5RNM4EX|          5|            0|          0|   Y|My father is 75 y...|   628|           false|
|R1H8QU9G9HLBWK|          5|           43|         49| 

In [19]:
# See number of reviews that are above average in length
review_above_mean = review_length_df.filter(review_length_df.length > 1290).count()
print(review_above_mean)

3


Evaluation of the Vine reviews looked at the total number of reviews, average rating, number of votes and length of reviews. The Vine reviews were removed from the total review population and examined separately.

The length of reviews is the strongest indicator that a review is legitimate and trustworthy. Second to this are the number of helpful votes.

Since this is such small result of Vine reviews for this category, further analysis isn't justified in this case, trying to see the distribution or look for outliers is not informative with a population size smaller than most sample sizes.

Of all of the sports reviews in this dataset, there are only 11 vine reviews, three of which have word counts above the mean. Spoofed or SPAM reviews are usually shorter in length. Some websites use rules of thumb like at least 20 words for a review to be accepted. All of these reviews exceeded that threshold by an order of magnitude. Furthermore, the shortest review was 265 words. A typed, full-length page has approximately 250 words. This is quite lengthy for a review and most any that are this long or longer are likely to be trustworthy reviews.

You can say these sports product fans aren't just fans of the product; they're fanatical about their reviews as well.
